# Imports

In [91]:
import tensorflow as tf
import tensorflow_text as tf_text
import random
from collections import Counter
import numpy as np
import tqdm as tqdm
import time
import re

# Open File


In [16]:
with open('bible.txt', 'r') as file:
    bible = file.read().replace('\n', ' ')

## Preprocess

In [119]:
def prep(text,k = 10000, splitter = "whitespace", with_dot = False):
    p = text.lower()
    
    if(with_dot):
        p = re.sub('[^A-Za-z. ]+', '', p)
    else: 
        p = re.sub('[^A-Za-z ]+', '', p)
    
    split_it = p.split()
    counter = Counter(split_it)
    most_occur = counter.most_common(k)

    p = ' '.join(p.split())

    if splitter == "whitespace":
        splitter = tf_text.WhitespaceTokenizer()
        p = splitter.split(p)  


    return p, split_it, most_occur


In [120]:
pbible, ogbible, most_occur = prep(bible)

# Splitter

In [124]:
print(pbible)
print(type(ogbible))

tf.Tensor([b'the' b'first' b'book' ... b'you' b'all' b'amen'], shape=(790017,), dtype=string)
<class 'list'>


In [60]:
def indexer(word, wlist):
    matched_indeces = []
    i = 0
    length = len(wlist)
    while i < length:
        if word == wlist[i]:
            matched_indeces.append(i)
        i += 1
    return matched_indeces

In [62]:
#print(indexer('the', ogbible))

In [92]:
def find_context(word, bible, k = 4):
    scope = []
    indexes = indexer(word, bible)
    index = indexes[random.randint(0, len(indexes)-1)]
    for i in range(int(k/2)):
        scope.append((word,bible[index-(i+1)]))
        scope.append((word,bible[index+(i+1)]))
    return scope

In [117]:
print(find_context('jesus', ogbible))

[('jesus', 'of'), ('jesus', 'christ'), ('jesus', 'appearing'), ('jesus', 'whom')]


In [ ]:
def create_dataset(bible, k = 10000):
